In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from chroma_store import chroma_store

embeddings_model = HuggingFaceEmbeddings()

loader = WebBaseLoader("https://www.lemonde.fr/international/article/2024/08/26/allemagne-apres-l-attentat-de-solingen-olaf-scholz-promet-de-durcir-tres-rapidement-la-legislation-sur-le-port-des-armes_6295608_3210.html")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
all_splits = text_splitter.split_documents(data)
chroma_store = chroma_store.from_documents(documents=all_splits, embedding=embeddings_model)


/home/unix/.pyenv/versions/3.11.9/envs/poc-rag/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


from langchain.callbacks.manager import CallbackManager
from langchain_core.callbacks import StreamingStdOutCallbackHandler
from langchain_community.llms import Ollama
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks in french. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. The answer must be in french.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)
llm = Ollama(model="mistral:latest", verbose=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | llm
    | StrOutputParser()
)

question = "Le chancelier allemand Olaf Scholz a qualifié quoi, le lundi 26 août ?"

docs = chroma_store.similarity_search(question)

# Run
chain.invoke({"context": docs, "question": question})

 Le chancelier allemand Olaf Scholz a qualifié un acte de « terrorisme contre nous » (il a qualifié un événement de terrorisme).

' Le chancelier allemand Olaf Scholz a qualifié un acte de « terrorisme contre nous » (il a qualifié un événement de terrorisme).'